In [17]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from mpp.models import Response
from datetime import datetime

import subprocess


In [18]:
# grab the clean text from the rds
with open('../../Response-Identification-Info/local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [19]:
response_id = 155033

response_url, response_cleaned, = session.query(
    Response.source_url,
    Response.cleaned_content
).filter(Response.id==response_id).first()

In [10]:
tfile = 'tmp/%s.xml' % response_id
with open(tfile, 'w') as f:
    f.write(response_cleaned)
    

In [20]:
script = '../../Response-Identification-Info/scripts/unique_identifier_cli.py'
cmd = 'python {0} -f {1} -u {2}'.format(script, tfile, response_url)

In [21]:
s = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = s.communicate()

In [22]:
stdout.split('\n')

['{"original_text": "http://oos.soest.hawaii.edu/las/UI.vm?dsid=dhw&varid=CRW_DHW-dhw", "extraction_type": "extract", "tag": "MI_Metadata/distributionInfo/MD_Distribution/distributor/MD_Distributor/distributorTransferOptions/MD_DigitalTransferOptions/onLine/CI_OnlineResource/linkage/URL", "match_type": "url", "potential_identifier": "http://oos.soest.hawaii.edu/las/UI.vm?dsid=dhw&varid=CRW_DHW-dhw"}',
 '{"original_text": "http://oos.soest.hawaii.edu/las/UI.vm", "extraction_type": "extract", "tag": "MI_Metadata/distributionInfo/MD_Distribution/distributor/MD_Distributor/distributorTransferOptions/MD_DigitalTransferOptions/onLine/CI_OnlineResource/linkage/URL", "match_type": "url", "potential_identifier": "http://oos.soest.hawaii.edu/las/UI.vm"}',
 '{"original_text": "http://oos.soest.hawaii.edu/dchart/index.html?dsetid=af866abc27c1df2b887ef6aac9a4e124", "extraction_type": "extract", "tag": "MI_Metadata/distributionInfo/MD_Distribution/distributor/MD_Distributor/distributorTransferOption

In [23]:
stderr

''